# HW 2: Car wash simulation

You've been contracted to write a simulation to help a local car wash with their
business plan. The machine they operate for the automated car wash has various
settings that affect the speed with which a car is washed and the amount of
water and soap that are consumed. The company wants to know how many customers
they can serve and how long they will have to wait in line.

The program specifications are as follows. The program has three input items:
1. The amount of time it takes to wash one car, in seconds
2. The probability that a new customer arrives during any given second
3. The total length of time to be simulated, in seconds

The program produces two outputs, based on its computations:
1. The number of customers served during the simulation time
2. The average waiting time for customers during the simulation, in seconds

In this assignment, you'll use a queue (and a few other objects) to manage the
simulation. The printer queue example in Miller and Ranum is an instructive
example. There are many comments and I have tried to make the instructions
detailed and clear. Please read carefully, and then *ask* if you are stuck!

There is a chat button in the top right of the Cocalc window (speech bubbles).
If you use it, I will see your message and respond.

### The queue of customers

All we really need to know about the customers is how long they wait in the
queue. So, when one arrives, we will represent them by a number called a
*timestamp*. The simulation works in seconds, so the timestamp will just be the
number of seconds that have already passed during the simulation before the
customer arrives. When a customer is removed from the queue (their car is
washed), we can then calculate the number of seconds they waited.

You'll use the `deque` object from Python's standard module `collections`.
There's no need to wrap it in a custom `Queue` class. If you restrict yourself
to using methods `deque.appendleft()` and `deque.pop()` then your deque is
functioning as a queue. Note that the `deque` class lacks our `.is_empty()`
method from class. Instead use `len()` just as you would for a list. (Puzzle:
how could we add this functionality to our homebrew `Queue` class?)

In [9]:
from collections import deque

### The `washer` object

Instances of the `washer` class simulate the machine that washes cars. Its
constructor will take one parameter \(in addition to `self`\): the number of
seconds needed to wash a car, `wash_time`. When the washer starts washing,
set its `time_until_done` attribute equal to `wash_time`.

Each time another second passes, the simulation program will indicate the
passage of one second for the washer. This is done by the instance method
`washer.one_second()`.

The washer needs to be able to tell the rest of the program whether it is
currently busy, and to start washing the next car in the queue. These are
done via member functions `washer.is_busy()` and `washer.wash()`.

Remember, the washer is abstract. It's just a timer, more or less. When you start it, it
resets its timer, `time_until_done`, to `wash_time`, and then counts down to 0.
When someone asks the washer if it's busy, it looks to see if its timer is at 0
and answers accordingly. Resist the temptation to create a `Car` class for the
washer to operate on. It doesn't need one.



In [10]:
class Washer:
    """The washer knows whether it is washing, and if it is, how long it will be until the
    next car can exit the waiting queue.
    """
    def __init__(self, wash_time):
        """Sets up a Washer instance. Make sure you know what the instance attributes should be!"""
        # YOUR CODE HERE
        self.wash_time = wash_time
        self.time_until_done = 0

    def is_busy(self):
        """Return True if the washer is currently washing (so no car can
        exit the queue yet) and False if not (the next car can be dequeued)."""
        # YOUR CODE HERE
        if self.time_until_done > 0:
            return True

    def start_washing(self):
        """Tell the washer to wash the car at the front of the
        queue by updating its attributes appropriately."""
        # YOUR CODE HERE
        self.time_until_done = self.wash_time

    def one_second(self):
        """Update the washer's attributes to reflect the passage of one second."""
        # YOUR CODE HERE
        self.time_until_done += -1
      

In [11]:
# Check that the Washer class does what it is supposed to:
from nose.tools import assert_equal
w = Washer(100)
assert_equal(w.wash_time, 100)
assert_equal(w.time_until_done, 0)
for key in vars(w): # makes sure you have not added more attributes
    assert(key in ('wash_time', 'time_until_done'))

w.time_until_done = 1
assert(w.is_busy())
w.time_until_done = 0
assert(not w.is_busy())

w.start_washing()
assert_equal(w.time_until_done, w.wash_time)

w.time_until_done = 10
w.one_second()
assert_equal(w.time_until_done, 9)

### Managing arrivals

We'll use the probability input to determine, during each simulated second,
whether or not a new customer arrives at the rear of the queue. This could be
done with a simple free-standing (i.e., not an instance method) function, but
the design choice here is to use a single instance of a custom class, called
`ArrivalGenerator`. The `ArrivalGenerator` class has a constructor that takes
one optional argument, the probability input of the program. (If no argument is
passed, the constructor uses the default value of `0.5`.) It has a single
instance method `ArrivalGenerator.query()` that returns either `True` or
`False`, with `True` occurring with probability given by the constructor's
argument. We'll use some helper functions from the `random` module to generate
these random occurrences.

In [12]:
import random

class ArrivalGenerator:
    def __init__(self, prob=0.5):
        """The ArrivalGenerator has one job: return True with probability `prob`.
        To do that, it needs to save the value of `prob`.
        """
        # YOUR CODE HERE
        self.probability = prob

    def query(self):
        """Return True with probability prob.""" 
        # There are many different ways you might use the `random` module's
        # functions to do it. If you want a hint, please ask, I don't want
        # you to get hung up on the math of probability too much. DND or other
        # tabletop RPG enthusiasts should know what to do immediately.
        # YOUR CODE HERE
        number = random.randrange(0,10001)
        if number <= self.probability*10000:
            return True
        else:
            return False


In [13]:
# Check that ArrivalGenerator does what it is supposed to do:
from nose.tools import assert_equal
a = ArrivalGenerator()
assert_equal(a.probability, 0.5)
a = ArrivalGenerator(0.9)
assert_equal(a.probability, 0.9)

arrivals_list = (a.query() for _ in range(1_000_000))
number_of_arrivals = sum([1 for x in arrivals_list if x])
# You should get something very close to 900000
assert(899000 < number_of_arrivals and number_of_arrivals < 901000)
# If it fails, but you think you are right, just try again. It is probabilistic,
# so could be false negative. But two false negatives is unlikely to happen.

### Tracking the average waiting time

To track the average waiting time, we'll use another custom class instance.
The class is called `AverageTracker`. Its only job is to compute the
average of a sequence of numbers. For example, we could send the values
234, 234, 908, and 279 into an `AverageTracker`. It could then tell us
that the average of these values is 413.75. It could also tell us that
so far it has processed 4 values. Thus we can use our `AverageTracker`
to keep track of the average waiting time *and* the number of customers
served during the simulation.

The `AverageTracker` has a constructor that prepares the `AverageTracker`
instance to accept a sequence of numbers. The instance receives one value
at a time through instance method `AverageTracker.next_value()`. Read carefully. 
The `AverageTracker` does **not** keep a list of all the values received so far, only
the running sum and number of values processed.

There are two instance methods to obtain info from the `AverageTracker`.
These are `AverageTracker.number_of_values()` and `AverageTracker.average()`.



In [14]:
class AverageTracker:
    def __init__(self):
        """The average tracker just needs to know the total of all numbers
        it has received so far and how many numbers it's received."""
        # YOUR CODE HERE
        self.sum_ = 0
        self.count = 0

    def next_value(self, val):
        """This method adds `val` to the total received so far and increments
        the number of values received."""
        # YOUR CODE HERE
        self.sum_ += val
        self.count += 1
        
    def average(self):
        """Return the average of all the values so far."""
        # YOUR CODE HERE
        avg = self.sum_ / self.count
        return avg

    def number_of_values(self):
        """Return the number of values received so far."""
        # YOUR CODE HERE
        return self.count


In [15]:
from nose.tools import assert_equal
import random
at = AverageTracker()
assert_equal(at.count, 0)
assert_equal(at.sum_, 0)
for key in vars(at):
    assert(key in ('count', 'sum_'))

at = AverageTracker()
random_value_list = [random.random() for _ in range(1000)]
for val in random_value_list:
    at.next_value(val)

assert_equal(at.count, len(random_value_list))
assert_equal(at.sum_, sum(random_value_list))
assert(at.average() == sum(random_value_list)/len(random_value_list))

### Pseudocode for main program

<ol style="list-style-type: upper-roman">
    <li>Initialize the input values for the program. These are arrival probability and simulation time.</li>
    <li>Initialize a <tt>Washer</tt> instance, making sure to supply a value for its parameter (how long it takes to wash a car).</li>
    <li>Initialize instances of <tt>ArrivalGenerator</tt>, <tt>AverageTracker</tt>, and <tt>deque</tt>.</li>
    <li>For each integer between 0 and the simulation time:</li>
    <ol style="list-style-type: upper-alpha">
        <li>Ask the <tt>ArrivalGenerator</tt> whether a new customer arrives during the current second. If so, enqueue the value of the current second.</li>
        <li>If the <tt>Washer</tt> is not busy and the queue is not empty:</li>
        <ol style="list-style-type: arabic">
            <li>Remove the next value from the queue. This is the arrival timestamp of the car about to get washed.</li>
            <li>Compute how long the next car had to wait, and send that value to the <tt>AverageTracker</tt>.</li>
            <li>Tell the <tt>Washer</tt> to start washing the next car in line.</li>
        </ol>
            <li>Tell the <tt>Washer</tt> another second has passed. (This is how the washer knows when it is no longer busy.)</li>
    </ol>
    <li>Now the simulation is done. Print a report including the simulation time and probability, number of cars washed, and average waiting time. Example:<br/>
        <tt>Simulation complete<br/>
        In 1000 seconds with probability 0.005: washed 23 cars with average waiting time 83.2 seconds.
        </tt>
    </li>
</ol>

### Read this carefully

Make sure you don't reinvent the wheel. A fully object-oriented, encapsulated style means that your main program does less work on its own. In this program, it should mostly coordinate the actions of various objects, who handle the details of the simulation as you've already written them. It does this by passing messages (values) between them. Set your objects up, then let them do their work.

### Testing your simulation

You should get realistic values with the probability, simulation time, and wash time that are provided.

In [16]:
from collections import deque

prob = 0.004
simulation_time = 6000
wash_time = 150
#YOUR CODE HERE       
ws = Washer(wash_time)
ag = ArrivalGenerator(prob)
at = AverageTracker()
dq = deque()
for current_second in range(simulation_time+1):
    if ag.query() == True:
        dq.append(current_second)
    if len(dq) != 0 and not ws.is_busy():
        last_car_washed = dq.popleft()
        at.next_value(current_second - last_car_washed)
        ws.start_washing()
    ws.one_second()
sm = "Simulation Complete"
ms = f"In {simulation_time} seconds with probability {prob}: washed {at.number_of_values()} cars with average waiting time {at.average()} seconds."
print(sm)
print(ms)
    



Simulation Complete
In 6000 seconds with probability 0.004: washed 21 cars with average waiting time 160.76190476190476 seconds.


### SIMULATE

Run your simulation ten thousand times (it will take a minute or two). Store the
results in two lists. Each run of the simulation produces a count and an
average. After each run, capture these values. Append the count to the list of
counts and the average to the list of averages. Return the two lists at once,
like this:

```python
return counts, averages
```



In [40]:
def one_thousand_runs():
    """
    Get lists of one thousand counts and averages from the simulator.
    Return the lists like this:
    return counts, averages
    """
    # YOUR CODE HERE
    counts = []
    averages = []
    for runs in range(10001):
        prob = 0.004
        simulation_time = 6000
        wash_time = 150   
        ws = Washer(wash_time)
        ag = ArrivalGenerator(prob)
        at = AverageTracker()
        dq = deque()
        for current_second in range(simulation_time+1):
            if ag.query() == True:
                dq.append(current_second)
            if len(dq) != 0 and not ws.is_busy():
                last_car_washed = dq.popleft()
                at.next_value(current_second - last_car_washed)
                ws.start_washing()
            ws.one_second()
        counts.append(at.number_of_values())
        averages.append(at.average())
    return counts, averages



In [42]:
one_thousand_runs()

([18,
  25,
  27,
  19,
  30,
  23,
  22,
  21,
  29,
  21,
  18,
  16,
  21,
  22,
  29,
  12,
  27,
  21,
  23,
  19,
  32,
  22,
  27,
  34,
  27,
  16,
  25,
  26,
  26,
  30,
  17,
  23,
  20,
  20,
  25,
  26,
  35,
  12,
  21,
  25,
  22,
  20,
  21,
  27,
  27,
  17,
  26,
  22,
  28,
  20,
  30,
  22,
  20,
  24,
  20,
  24,
  20,
  23,
  15,
  24,
  23,
  34,
  18,
  24,
  23,
  30,
  30,
  20,
  16,
  29,
  14,
  20,
  32,
  32,
  14,
  19,
  24,
  27,
  25,
  23,
  25,
  25,
  26,
  24,
  20,
  29,
  25,
  25,
  19,
  20,
  26,
  22,
  31,
  24,
  25,
  15,
  22,
  19,
  37,
  22,
  30,
  34,
  26,
  12,
  20,
  22,
  22,
  20,
  27,
  24,
  22,
  25,
  22,
  25,
  18,
  15,
  30,
  22,
  24,
  18,
  24,
  26,
  22,
  21,
  24,
  27,
  22,
  28,
  31,
  20,
  32,
  26,
  17,
  30,
  19,
  19,
  28,
  17,
  24,
  20,
  24,
  28,
  25,
  22,
  24,
  20,
  23,
  26,
  22,
  23,
  26,
  21,
  21,
  26,
  27,
  24,
  25,
  27,
  31,
  26,
  28,
  22,
  23,
  26,
  34,
  23,
  16

To see a graph of your simulated results, run the next cell. Make sure you answer the essay question below it.



In [45]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-whitegrid')
plt.plot(counts, averages, '.', color='black')
plt.xlabel("counts")
plt.ylabel("average wait time");

/tmp/ipykernel_523/486338789.py:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


NameError: name 'counts' is not defined

### Reflection essay (100-200 words, please)

What did you learn during this assignment? How could the assignment have been
better? What would you keep the same about it? And finally, how would you
improve the simulator?

# Put your essay in this cell

In [38]:
"It was very interesting to see object oriented programing in practice using an example that could be used in the real world. It took some time at first to get familiarized with how the functions would work which each other and understand the concept of self. It took me more time than necessary to finish this homework as it had a bug in the washer class which even though had passed all tests, made the output of the simulation not come out as it was suposed to.In the end, was just one single signal that was causing this. It was a little bit stressful but important lesson to learn, and that it will not be the first time it will happen. Overall, I enjoyed the assignement, since I though it brought more programing skills and dealing with problems than learning in theory."

'It was very interesting to see object oriented programing in practice using an example that could be used in the real world. It took some time at first to get familiarized with how the functions would work which each other and understand the concept of self. It took me more time than necessary to finish this homework as it had a bug in the washer class which even though had passed all tests, made the output of the simulation not come out as it was suposed to.In the end, was just one single signal that was causing this. It was a little bit stressful but important lesson to learn, and that it will not be the first time it will happen. Overall, I enjoyed the assignement, since I though it brought more programing skills and dealing with problems than learning in theory.'